Generating code for a model:

In [1]:
import pytorch_composer

input_dim = [4,3,32,32]
# A random sequence of neural network layers. Any positive integer shoud be a valid dimension arguement:
sequence = [
    ["Conv2d", 6],
    ["Reshape",(4,32,32)],
    ["MaxPool2d", 2],
    ["Linear", 52],
    ["RNN",47],
    ["Relu"],
    ["MaxPool2d", 2],
    ["Linear",43],
    ["MaxPool2d", 2],
    ["RNN",38],
    ["Relu"],
    ["Flat"],
    ["Linear",38],
]

model = pytorch_composer.Model(sequence, input_dim)
model


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d1 = nn.Conv2d(3, 6, 3)
        self.adaptiveavgpool2d1 = nn.AdaptiveAvgPool2d((32, 32))
        self.maxpool2d1 = nn.MaxPool2d(2, stride=2)
        self.linear1 = nn.Linear(16, 52)
        self.rnn1 = nn.RNN(832, 47)
        self.adaptiveavgpool1d1 = nn.AdaptiveAvgPool1d(49)
        self.linear2 = nn.Linear(3, 43)
        self.rnn2 = nn.RNN(21, 38)
        self.linear3 = nn.Linear(38, 38)

    def forward(self, x, h1, h2):
        # Input dimensions : (4, 3, 32, 32)
        # Convolution layer (2d): (4, 3, 32, 32) -> (4, 6, 30, 30)
        x = self.conv2d1(x)
        x = x.view(1, 4, 6, 900)
        # Resizing with adaptive average pooling: (1, 4, 6, 900) -> (1, 4, 32, 32)
        x = self.adaptiveavgpool2d1(x)
        x = x.view(4, 32, 32)
        x = x.view(4, 1, 32, 32)
        # Pooling layer (2d max): (4, 1, 32, 32) -> (4, 1, 16, 16)
        x = self.maxpool2d1(x)
        # Linear

Generating a training loop:

In [2]:
loop = pytorch_composer.TrainingLoop.from_model(model)
loop


# Define a Loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# Training
for epoch in range(2):  # loop over the dataset multiple times
    h1, h2 = net.initHidden()

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, h1, h2 = net(inputs, h1, h2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        h1 = h1.data
        h2 = h2.data

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        torch.save(net.state_dic

Using the generated code:

In [3]:
import torch

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


# Load and normalize the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d1 = nn.Conv2d(3, 6, 3)
        self.adaptiveavgpool2d1 = nn.AdaptiveAvgPool2d((32, 32))
        self.maxpool2d1 = nn.MaxPool2d(2, stride=2)
        self.linear1 = nn.Linear(16, 52)
        self.rnn1 = nn.RNN(832, 47)
        self.adaptiveavgpool1d1 = nn.AdaptiveAvgPool1d(49)
        self.linear2 = nn.Linear(3, 43)
        self.rnn2 = nn.RNN(21, 38)
        self.linear3 = nn.Linear(38, 38)

    def forward(self, x, h1, h2):
        # Input dimensions : (4, 3, 32, 32)
        # Convolution layer (2d): (4, 3, 32, 32) -> (4, 6, 30, 30)
        x = self.conv2d1(x)
        x = x.view(1, 4, 6, 900)
        # Resizing with adaptive average pooling: (1, 4, 6, 900) -> (1, 4, 32, 32)
        x = self.adaptiveavgpool2d1(x)
        x = x.view(4, 32, 32)
        x = x.view(4, 1, 32, 32)
        # Pooling layer (2d max): (4, 1, 32, 32) -> (4, 1, 16, 16)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 1, 16, 16) -> (4, 1, 16, 52)
        x = self.linear1(x)
        x = x.view(4, 1, 832)
        # Recurrent layer: (4, 1, 832) -> (4, 1, 47)
        x, h1 = self.rnn1(x, h1)
        x = F.relu(x)
        x = x.view(4, 1, 47)
        # Resizing with adaptive average pooling: (4, 1, 47) -> (4, 1, 49)
        x = self.adaptiveavgpool1d1(x)
        x = x.view(4, 1, 7, 7)
        # Pooling layer (2d max): (4, 1, 7, 7) -> (4, 1, 3, 3)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 1, 3, 3) -> (4, 1, 3, 43)
        x = self.linear2(x)
        # Pooling layer (2d max): (4, 1, 3, 43) -> (4, 1, 1, 21)
        x = self.maxpool2d1(x)
        x = x.view(4, 1, 21)
        # Recurrent layer: (4, 1, 21) -> (4, 1, 38)
        x, h2 = self.rnn2(x, h2)
        x = F.relu(x)
        x = x.view(4, 38)
        # Linear layer: (4, 38) -> (4, 38)
        x = self.linear3(x)
        # Output dimensions : (4, 38)
        return x, h1, h2

    def initHidden(self):
        h1 = torch.zeros(1, 1, 47)
        h2 = torch.zeros(1, 1, 38)
        return h1, h2
    
# Define a Loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# Training
for epoch in range(2):  # loop over the dataset multiple times
    h1, h2 = net.initHidden()

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, h1, h2 = net(inputs, h1, h2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        h1 = h1.data
        h2 = h2.data

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        torch.save(net.state_dict(), 'model.pt')

print('Finished Training')

[1,  2000] loss: 2.495
[1,  4000] loss: 2.328
[1,  6000] loss: 2.322
[1,  8000] loss: 2.318
[1, 10000] loss: 2.319
[1, 12000] loss: 2.317
[2,  2000] loss: 2.316
[2,  4000] loss: 2.315
[2,  6000] loss: 2.314
[2,  8000] loss: 2.295
[2, 10000] loss: 2.187
[2, 12000] loss: 2.116
Finished Training
